# Correr experimentos
En este archivo está el código para correr los experimentos y escribir los resultados en archivos CSV.
> Los archivos se guardan en la carpeta _resultados_.

In [91]:
import math, subprocess
import pandas as pd
import numpy as np
from IPython.display import display, clear_output
import re

A continuación leemos los datasets en dataframes de Pandas.

In [92]:
def leer_instancia(path_instancia):
    with open(path_instancia, "r") as f:
        return f.read();

df_tsplib = pd.read_csv("experimentos/tsplib.csv");
df_nuestros = pd.read_csv("experimentos/nuestros.csv");
df_mismotamanio = pd.read_csv("experimentos/mismo_tamanio.csv");

In [93]:
def correr_experimento(*args):
    # Crear proceso para ejecutar el codigo.
    process = subprocess.Popen(["../main", *args], stderr=subprocess.PIPE, stdout=subprocess.PIPE, stdin=subprocess.PIPE, universal_newlines = True)

    # Correr experimento.
    exit_code = process.wait()

    # Verificar que el proceso no fallo.
    if exit_code != 0: raise(F"Hubo un error en la experimentacion para el algoritmo: {args}.")
    # Leer salida de STDERR con los tiempos de ejecucion de cada metodo.
    tiempo_de_ejecucion = float(process.stderr.read());
    [tamanio, costo_resultado] = process.stdout.read().split('\n')[0].split(" ");

    process.stdin.close();
    process.stdout.close();
    process.stderr.close();
    
    return [tiempo_de_ejecucion, tamanio, costo_resultado];

## Corremos los experimentos
Vamos a guardar una tabla con las ejecuciones y sus respectivos tiempos.

In [94]:
experimentos = [];

## Experimento 1
Correr Heurisitica Golosa sobre todas las instancias de tsplib.

In [52]:
for index, row in df_tsplib.iterrows():
    experimentos.append([row["instancia"], row["ruta_instancia"], "H-CG"]);

In [95]:
for index, row in df_nuestros.iterrows():
    experimentos.append([row["instancia"], row["ruta_instancia"], "H-CG"]);

In [81]:
for index, row in df_mismotamanio.iterrows():
    experimentos.append([row["instancia"], row["ruta_instancia"], "H-CG"]);

## Experimento 2
Correr Heurisitica AGM sobre todas las instancias de tsplib.

In [53]:
for index, row in df_tsplib.iterrows():
    experimentos.append([row["instancia"], row["ruta_instancia"], "H-AGM"]);

In [96]:
for index, row in df_nuestros.iterrows():
    experimentos.append([row["instancia"], row["ruta_instancia"], "H-AGM"]);

In [82]:
for index, row in df_mismotamanio.iterrows():
    experimentos.append([row["instancia"], row["ruta_instancia"], "H-AGM"]);

## Experimento 3
Correr Busqueda Local sobre todas las instancias de tsplib.

In [54]:
for index, row in df_tsplib.iterrows():
    experimentos.append([row["instancia"], row["ruta_instancia"], "BL"]);

In [97]:
for index, row in df_nuestros.iterrows():
    experimentos.append([row["instancia"], row["ruta_instancia"], "BL"]);

In [83]:
for index, row in df_mismotamanio.iterrows():
    experimentos.append([row["instancia"], row["ruta_instancia"], "BL"]);

## Experimento 4
Correr Tabu Search sobre todas las instancias de tsplib.

In [55]:
for tipo_memoria in ['ciclos']:
    for T in [10, 100, 1000]:
        for iteraciones in [100, 500, 1000]:
            for index, row in df_tsplib.iterrows():
                if (int(re.findall(r'\d+', row["instancia"])[0]) > 100): continue
                experimentos.append([row["instancia"], row["ruta_instancia"], "BL-Tabu", tipo_memoria, str(iteraciones), str(T)])

In [98]:
for index, row in df_nuestros.iterrows():
    experimentos.append([row["instancia"], row["ruta_instancia"], "BL-Tabu", "ciclos", str(1000), str(1000)])

In [84]:
for index, row in df_mismotamanio.iterrows():
    experimentos.append([row["instancia"], row["ruta_instancia"], "BL-Tabu"]);

## Ejecutar los experimentos y guardar los resultados en un archivo CSV.
Este paso puede tardar unos minutos hasta terminar de ejecutarse.

In [87]:
columnas = ["instancia", "metodo", "tiempo", 'tamaño', 'costo', 'tipo_memoria', 'iteraciones', 'T'];
filas = [];
numero = 1
T = 5 # Numero de veces que se ejecuta cada experimento (para mayor fidelidad del tiempo).
for experimento in experimentos:
    # Voy mostrando que experimento se esta ejecutando.
    clear_output(wait=True)
    display('Experimento: ' + str(numero) + "/" + str(len(experimentos)))
    numero += 1
    
    # Ejecutamos el experimento T veces y obtenemos la mediana.
    tiempos = []
    tamanio_instancia = 0
    costo_resultado = 0
    T = 1 if experimento[2] == 'BL-Tabu' else 5
    for i in range(0, T):
        [tiempo, tamanio, costo] = correr_experimento(experimento[2], experimento[1], *experimento[3:])
        tiempos.append(tiempo);
        tamanio_instancia = tamanio
        costo_resultado = costo
    tiempo = np.median(tiempos);
    if len(experimento) == 3:
        experimento.append(None)
        experimento.append(None)
        experimento.append(None)
    filas.append([experimento[0], experimento[2], tiempo, tamanio, costo, *experimento[3:]]);
    print([experimento[0], experimento[2], tiempo, tamanio, costo, *experimento[3:]])
    df_resultado = pd.DataFrame(filas, columns=columnas);
    df_resultado.to_csv("resultados/resultado-fijos.csv", index=False, header=True);


'Experimento: 1/396'

TypeError: expected str, bytes or os.PathLike object, not NoneType